### KNN 훈련 및 저장

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

class StudentImprovementAdvisor:
    def __init__(self, df, grade_column='grade', k=10):
        """
        KNN 기반 학생 특성 개선 분석
        
        Parameters:
        df: 학생 데이터프레임 (전처리 완료된 상태)
        grade_column: 성적 컬럼명
        k: 비교할 유사 학생 수
        """
        self.df = df.copy()
        self.grade_column = grade_column
        self.k = k
        self.scaler = StandardScaler()
        
        # 컬럼 분류
        self.unchangeable_columns = ['fromCity', 'sex', 'age', 'Pedu']  # 바꿀 수 없는 특성
        self.decrease_only_columns = ['traveltime', 'freetime', 'goout', 'alc', 'absences']  # 줄이는 것만 권장
        self.increase_only_columns = ['studytime']  # 늘리는 것만 권장
        self.binary_increase_columns = ['activities', 'internet']  # 이진 데이터, 1로만 권장
        
        # 성적 컬럼을 제외하고 분석 가능한 특성들만 선택
        all_feature_columns = [col for col in df.columns 
                              if col != grade_column and df[col].dtype in ['int64', 'float64']]
        
        # 바꿀 수 없는 컬럼은 제외
        self.feature_columns = [col for col in all_feature_columns 
                               if col not in self.unchangeable_columns]
        
        # 바로 데이터 준비
        X = self.df[self.feature_columns].values
        self.X_scaled = self.scaler.fit_transform(X)
        
        # KNN 모델 학습
        self.knn = NearestNeighbors(n_neighbors=self.k+1, metric='euclidean')  # +1은 자기 자신 제외용
        self.knn.fit(self.X_scaled)
        
        print(f"분석 대상 특성: {self.feature_columns}")
        print(f"성적 컬럼: {grade_column}")
        print(f"분석 준비 완료: {len(self.df)}명의 학생 데이터")
        
    def analyze_student_improvement(self, student_idx):
        """
        특정 학생의 개선점 분석
        
        Parameters:
        student_idx: 분석할 학생의 인덱스 (DataFrame의 index)
        """
        if student_idx not in self.df.index:
            print(f"학생 인덱스 {student_idx}를 찾을 수 없습니다.")
            return None
            
        # 현재 학생 정보
        target_student = self.df.loc[student_idx]
        target_grade = target_student[self.grade_column]
        target_features = target_student[self.feature_columns]
        
        print(f"\n=== {target_student.get('name', f'학생_{student_idx}')} 분석 ===")
        print(f"현재 성적: {target_grade}")
        
        # 데이터프레임에서 해당 학생의 위치 찾기
        df_idx = self.df.index.get_loc(student_idx)
        target_scaled = self.X_scaled[df_idx].reshape(1, -1)
        
        # 유사한 학생들 찾기
        distances, indices = self.knn.kneighbors(target_scaled)
        
        # 자기 자신 제외
        similar_indices = indices[0][1:]  # 첫 번째는 자기 자신
        similar_distances = distances[0][1:]
        
        # 유사한 학생들 중 성적이 더 좋은 학생들만 필터링
        better_students = []
        for idx, dist in zip(similar_indices, similar_distances):
            actual_idx = self.df.index[idx]  # DataFrame의 실제 인덱스
            similar_student = self.df.iloc[idx]
            if similar_student[self.grade_column] > target_grade:
                better_students.append({
                    'index': actual_idx,
                    'data': similar_student,
                    'distance': dist
                })
        
        if not better_students:
            print("유사한 특성을 가진 성적 상위 학생을 찾을 수 없습니다.")
            print("더 넓은 범위(k 값 증가)로 다시 분석해보세요.")
            return None
            
        # 성적이 더 좋은 유사 학생들의 평균 특성 계산
        better_df = pd.DataFrame([student['data'] for student in better_students])
        avg_better_features = better_df[self.feature_columns].mean()
        avg_better_grade = better_df[self.grade_column].mean()
        
        print(f"\n성적 상위 유사 학생 {len(better_students)}명 (평균 성적: {avg_better_grade:.1f})")
        
        # 개선점 분석 (컬럼별 제약사항 적용)
        improvements = {}
        valid_improvements = {}  # 실제 권장할 수 있는 개선사항만
        
        for feature in self.feature_columns:
            current_value = target_features[feature]
            better_avg = avg_better_features[feature]
            difference = better_avg - current_value
            improvement_ratio = (difference / current_value * 100) if current_value != 0 else 0
            
            improvements[feature] = {
                'current': current_value,
                'target_avg': better_avg,
                'difference': difference,
                'improvement_ratio': improvement_ratio
            }
            
            # 컬럼별 제약사항 확인
            should_recommend = False
            recommendation_type = ""
            
            if feature in self.decrease_only_columns:
                if difference < 0:  # 줄여야 하는 경우만 권장
                    should_recommend = True
                    recommendation_type = "감소"
            elif feature in self.increase_only_columns:
                if difference > 0:  # 늘려야 하는 경우만 권장
                    should_recommend = True
                    recommendation_type = "증가"
            elif feature in self.binary_increase_columns:
                if current_value == 0 and better_avg > 0.5:  # 현재 0이고 상위학생들이 주로 1인 경우
                    should_recommend = True
                    recommendation_type = "참여"
            else:  # 일반 특성 (Pedu 등)
                if abs(difference) > 0.1:  # 의미있는 차이가 있는 경우
                    should_recommend = True
                    recommendation_type = "증가" if difference > 0 else "감소"
            
            if should_recommend:
                improvements[feature]['recommendation_type'] = recommendation_type
                valid_improvements[feature] = improvements[feature]
        
        # 개선 우선순위 (유효한 개선사항만, 개선 비율 기준)
        sorted_improvements = sorted(valid_improvements.items(), 
                                   key=lambda x: abs(x[1]['improvement_ratio']), 
                                   reverse=True)
        
        print(f"\n📈 개선 권장사항 (우선순위 순):")
        print("="*60)
        
        if not sorted_improvements:
            print("현재 상태가 이미 우수하거나 개선 가능한 특성이 없습니다!")
        else:
            for i, (feature, data) in enumerate(sorted_improvements, 1):
                rec_type = data['recommendation_type']
                
                print(f"{i}. {feature} ({rec_type})")
                print(f"   현재: {data['current']:.1f}")
                
                if feature in self.binary_increase_columns:
                    if data['current'] == 0:
                        print(f"   권장: 참여하세요!")
                    else:
                        print(f"   권장: 계속 참여하세요!")
                elif rec_type == "감소":
                    print(f"   목표: {data['target_avg']:.1f} (현재보다 {abs(data['difference']):.1f} 감소)")
                elif rec_type == "증가":
                    print(f"   목표: {data['target_avg']:.1f} (현재보다 {data['difference']:.1f} 증가)")
                
                print(f"   개선 필요도: {abs(data['improvement_ratio']):.1f}%")
                print()
        
        return {
            'target_student': target_student,
            'better_students': better_students,
            'improvements': improvements,
            'sorted_improvements': sorted_improvements
        }
    

In [3]:
df = pd.DataFrame(pd.read_csv('../data/fkillerML_data.csv'))

# 1. 분석기 초기화 (바로 분석 준비 완료)
advisor = StudentImprovementAdvisor(df, grade_column='grade', k=30)

# 2. 특정 학생 분석 (예: 인덱스 0인 학생)
result = advisor.analyze_student_improvement(50)

분석 대상 특성: ['traveltime', 'studytime', 'activities', 'internet', 'freetime', 'goout', 'alc', 'absences']
성적 컬럼: grade
분석 준비 완료: 315명의 학생 데이터

=== 학생_50 분석 ===
현재 성적: 12.666666666666666

성적 상위 유사 학생 10명 (평균 성적: 14.7)

📈 개선 권장사항 (우선순위 순):
1. alc (감소)
   현재: 2.5
   목표: 1.5 (현재보다 1.0 감소)
   개선 필요도: 40.0%

2. traveltime (감소)
   현재: 3.0
   목표: 2.2 (현재보다 0.8 감소)
   개선 필요도: 26.7%

3. studytime (증가)
   현재: 2.0
   목표: 2.1 (현재보다 0.1 증가)
   개선 필요도: 5.0%

4. goout (감소)
   현재: 3.0
   목표: 2.9 (현재보다 0.1 감소)
   개선 필요도: 3.3%



In [4]:
result.keys(), result['sorted_improvements']

(dict_keys(['target_student', 'better_students', 'improvements', 'sorted_improvements']),
 [('alc',
   {'current': np.float64(2.5),
    'target_avg': np.float64(1.5),
    'difference': np.float64(-1.0),
    'improvement_ratio': np.float64(-40.0),
    'recommendation_type': '감소'}),
  ('traveltime',
   {'current': np.float64(3.0),
    'target_avg': np.float64(2.2),
    'difference': np.float64(-0.7999999999999998),
    'improvement_ratio': np.float64(-26.66666666666666),
    'recommendation_type': '감소'}),
  ('studytime',
   {'current': np.float64(2.0),
    'target_avg': np.float64(2.1),
    'difference': np.float64(0.10000000000000009),
    'improvement_ratio': np.float64(5.000000000000004),
    'recommendation_type': '증가'}),
  ('goout',
   {'current': np.float64(3.0),
    'target_avg': np.float64(2.9),
    'difference': np.float64(-0.10000000000000009),
    'improvement_ratio': np.float64(-3.333333333333336),
    'recommendation_type': '감소'})])

In [5]:
df.columns

Index(['fromCity', 'sex', 'age', 'Pedu', 'traveltime', 'studytime',
       'activities', 'internet', 'freetime', 'goout', 'alc', 'absences',
       'grade'],
      dtype='object')

In [8]:
advisor.feature_columns

['traveltime',
 'studytime',
 'activities',
 'internet',
 'freetime',
 'goout',
 'alc',
 'absences']

### 훈련된 KNN 저장

In [ ]:
import joblib

# 저장
joblib.dump(advisor.knn, "knn_model_v1.0.0.pkl")

['knn_model_v1.0.0.pkl']

### Scaler 저장

In [7]:
import joblib

joblib.dump(advisor.scaler, "scaler.pkl")

['scaler.pkl']